In [2]:
class Stock:
    def __init__(self, symbol, date, open_, high, low, close, volume):
        self.symbol = symbol
        self.date = date
        self.open = open_
        self.high = high
        self.low = low
        self.close = close
        self.volume = volume
        
    def as_dict(self):
        return dict(symbol=self.symbol,
                   date=self.date,
                   open=self.open,
                   high=self.high,
                   low=self.low,
                   close=self.close,
                   volume=self.volume)
        


In [3]:
class Trade:
    def __init__(self, symbol, timestamp, order, price, volume, commission):
        self.symbol = symbol
        self.timestamp = timestamp
        self.order = order
        self.price = price
        self.commission = commission
        self.volume = volume
        
    def as_dict(self):
        return dict(symbol=self.symbol,
                   timestamp=self.timestamp,
                   order=self.order,
                   price=self.price,
                   commission=self.commission,
                   volume=self.volume)  
        

In [4]:
from datetime import date, datetime
from decimal import Decimal

activity = {
    "quotes": [
        Stock('TSLA', date(2018, 11, 22), 
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607),
        Stock('AAPL', date(2018, 11, 22), 
              Decimal('176.66'), Decimal('177.25'), Decimal('176.64'), Decimal('176.78'), 3_699_184),
        Stock('MSFT', date(2018, 11, 22), 
              Decimal('103.25'), Decimal('103.48'), Decimal('103.07'), Decimal('103.11'), 4_493_689)
    ],
    
    "trades": [
        Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.25'), 100, Decimal('9.99')),
        Trade('AAPL', datetime(2018, 11, 22, 10, 30, 5), 'sell', Decimal('177.01'), 20, Decimal('9.99'))
    ]
}

In [5]:
from json import JSONEncoder, dumps

In [11]:
class CustomEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Stock):
            return obj.as_dict()
        elif isinstance(obj, Trade):
            return obj.as_dict()
        else:
            super().default(obj)

In [12]:
dumps(activity, cls = CustomEncoder)

TypeError: Object of type date is not JSON serializable

In [16]:
class CustomEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Stock):
            result =  obj.as_dict()
            result['date'] = result['date'].strftime('%Y-%m-%d')
            return result
        elif isinstance(obj, Trade):
            result= obj.as_dict()
            result['timestamp']= result['timestamp'].strftime('%Y-%m-%dT%H:%M:%S')
        else:
            super().default(obj)

In [17]:
dumps(activity, cls = CustomEncoder)

TypeError: Object of type Decimal is not JSON serializable

In [18]:
class CustomEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Stock) or isinstance(obj, Trade):
            return obj.as_dict()
        elif isinstance(obj, date):
            return obj.strftime('%Y-%m-%d')
        elif isinstance(obj, datetime):
            return obj.strftime('%Y-%m-%dT%H:%M:%S')
        elif isinstance(obj, Decimal):
            return str(obj)
        else:
            super().default(obj)

In [19]:
encoded = dumps(activity, cls = CustomEncoder, indent =2)

In [20]:
print(encoded)

{
  "quotes": [
    {
      "symbol": "TSLA",
      "date": "2018-11-22",
      "open": "338.19",
      "high": "338.64",
      "low": "337.60",
      "close": "338.19",
      "volume": 365607
    },
    {
      "symbol": "AAPL",
      "date": "2018-11-22",
      "open": "176.66",
      "high": "177.25",
      "low": "176.64",
      "close": "176.78",
      "volume": 3699184
    },
    {
      "symbol": "MSFT",
      "date": "2018-11-22",
      "open": "103.25",
      "high": "103.48",
      "low": "103.07",
      "close": "103.11",
      "volume": 4493689
    }
  ],
  "trades": [
    {
      "symbol": "TSLA",
      "timestamp": "2018-11-22",
      "order": "buy",
      "price": "338.25",
      "commission": "9.99",
      "volume": 100
    },
    {
      "symbol": "AAPL",
      "timestamp": "2018-11-22",
      "order": "sell",
      "price": "177.01",
      "commission": "9.99",
      "volume": 20
    }
  ]
}


In [24]:
class CustomEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Stock) or isinstance(obj, Trade):
            result =  obj.as_dict()
            result['object'] = obj.__class__.__name__
            return result
        elif isinstance(obj, date):
            return obj.strftime('%Y-%m-%d')
        elif isinstance(obj, datetime):
            return obj.strftime('%Y-%m-%dT%H:%M:%S')
        elif isinstance(obj, Decimal):
            return str(obj)
        else:
            super().default(obj)

In [25]:
encoded = dumps(activity, cls = CustomEncoder, indent =2)

In [26]:
print(encoded)

{
  "quotes": [
    {
      "symbol": "TSLA",
      "date": "2018-11-22",
      "open": "338.19",
      "high": "338.64",
      "low": "337.60",
      "close": "338.19",
      "volume": 365607,
      "object": "Stock"
    },
    {
      "symbol": "AAPL",
      "date": "2018-11-22",
      "open": "176.66",
      "high": "177.25",
      "low": "176.64",
      "close": "176.78",
      "volume": 3699184,
      "object": "Stock"
    },
    {
      "symbol": "MSFT",
      "date": "2018-11-22",
      "open": "103.25",
      "high": "103.48",
      "low": "103.07",
      "close": "103.11",
      "volume": 4493689,
      "object": "Stock"
    }
  ],
  "trades": [
    {
      "symbol": "TSLA",
      "timestamp": "2018-11-22",
      "order": "buy",
      "price": "338.25",
      "commission": "9.99",
      "volume": 100,
      "object": "Trade"
    },
    {
      "symbol": "AAPL",
      "timestamp": "2018-11-22",
      "order": "sell",
      "price": "177.01",
      "commission": "9.99",
      "v

In [31]:
def decode_stock(d):
    s = Stock(d['symbol'],
             datetime.strptime(d['date'], '%Y-%m-%d').date(),
             Decimal(d['open']),
             Decimal(d['high']),
             Decimal(d['low']),
             Decimal(d['close']),
             d['volume']
             )
    return s

In [32]:
d = {
      "symbol": "MSFT",
      "date": "2018-11-22",
      "open": "103.25",
      "high": "103.48",
      "low": "103.07",
      "close": "103.11",
      "volume": 4493689,
      "object": "Stock"
    }

In [33]:
s =decode_stock(d)

In [34]:
type(s)

__main__.Stock

In [35]:
vars(s)

{'symbol': 'MSFT',
 'date': datetime.date(2018, 11, 22),
 'open': Decimal('103.25'),
 'high': Decimal('103.48'),
 'low': Decimal('103.07'),
 'close': Decimal('103.11'),
 'volume': 4493689}

In [39]:
def decode_trade(d):
    s = Trade(d['symbol'],
              datetime.strptime(d['timestamp'], '%Y-%m-%dT%H:%M:%S'),
              d['order'],
              Decimal(d['price']),
              d['volume'],
              Decimal(d['commission'])
             )
    return s

In [40]:
t = decode_trade({
      "symbol": "AAPL",
      "timestamp": "2018-11-22T10:30:05",
      "order": "sell",
      "price": "177.01",
      "commission": "9.99",
      "volume": 20,
      "object": "Trade"
    })

In [41]:
type(t)

__main__.Trade

In [42]:
vars(t)

{'symbol': 'AAPL',
 'timestamp': datetime.datetime(2018, 11, 22, 10, 30, 5),
 'order': 'sell',
 'price': Decimal('177.01'),
 'commission': Decimal('9.99'),
 'volume': 20}

In [43]:
def decode_finan(d):
    object_type = d.get('object', None)
    if object_type == 'Stock':
        return decode_stock(d)
    elif object_type == 'Trade':
        return decode_trade(d)
    return d

In [52]:
decode_finan(d = {
      "symbol": "MSFT",
      "date": "2018-11-22T10:30:05",
      "open": "103.25",
      "high": "103.48",
      "low": "103.07",
      "close": "103.11",
      "volume": 4493689,
      "object": "Stock"
    })

ValueError: unconverted data remains: T10:30:05

In [53]:
decode_finan({'a':1})

{'a': 1}

In [54]:
from json import JSONDecoder, loads

In [55]:
class CustomDecoder(JSONDecoder):
    def decode(self, arg):
        data = loads(arg)
        return self.parse_finan(data)
    def parse_finan(self, obj):
        if isinstance(obj, dict):
            obj = decode_finan(obj)
            if isinstance(obj, dict):
                for k, v in obj.items():
                    obj[k]=self.parse_finan(v)
        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index] = self.parse_finan(item)
        return obj

In [56]:
print(encoded)

{
  "quotes": [
    {
      "symbol": "TSLA",
      "date": "2018-11-22",
      "open": "338.19",
      "high": "338.64",
      "low": "337.60",
      "close": "338.19",
      "volume": 365607,
      "object": "Stock"
    },
    {
      "symbol": "AAPL",
      "date": "2018-11-22",
      "open": "176.66",
      "high": "177.25",
      "low": "176.64",
      "close": "176.78",
      "volume": 3699184,
      "object": "Stock"
    },
    {
      "symbol": "MSFT",
      "date": "2018-11-22",
      "open": "103.25",
      "high": "103.48",
      "low": "103.07",
      "close": "103.11",
      "volume": 4493689,
      "object": "Stock"
    }
  ],
  "trades": [
    {
      "symbol": "TSLA",
      "timestamp": "2018-11-22",
      "order": "buy",
      "price": "338.25",
      "commission": "9.99",
      "volume": 100,
      "object": "Trade"
    },
    {
      "symbol": "AAPL",
      "timestamp": "2018-11-22",
      "order": "sell",
      "price": "177.01",
      "commission": "9.99",
      "v

In [57]:
decoded = loads(encoded, cls = CustomDecoder)

ValueError: time data '2018-11-22' does not match format '%Y-%m-%dT%H:%M:%S'